In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib.request
import zipfile
import os
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from datetime import datetime

2023-07-23 10:16:46.986218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 10:16:49.854459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!wget --no-check-certificate \
    "https://github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/archive/refs/heads/main.zip" \
    -O "/tmp/Arthritis-dataset.zip"



zip_ref = zipfile.ZipFile('/tmp/Arthritis-dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-07-23 10:16:57--  https://github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/archive/refs/heads/main.zip
Connecting to 10.99.0.130:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/zip/refs/heads/main [following]
--2023-07-23 10:16:57--  https://codeload.github.com/ShreyaJaiswal1604/Detecting_Arthritis_X-Ray_Images/zip/refs/heads/main
Connecting to 10.99.0.130:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/Arthritis-dataset.zip’

/tmp/Arthritis-data     [               <=>  ]  91.40M  31.9MB/s    in 2.9s    

2023-07-23 10:17:00 (31.9 MB/s) - ‘/tmp/Arthritis-dataset.zip’ saved [95845017]



In [3]:
# Define dataset paths
base_dir = '/tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')
log_dir = os.path.join(base_dir, 'logs/amad')

In [4]:
print("base_dir", base_dir)
print("train_dir", train_dir)
print("test_dir", test_dir)
print("val_dir", val_dir)
print("log_dir", log_dir)

base_dir /tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset
train_dir /tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset/train
test_dir /tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset/test
val_dir /tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset/val
log_dir /tmp/Detecting_Arthritis_X-Ray_Images-main/Image_Datasets/Original_Kaggle_Binary_Dataset/logs/amad


In [5]:
# Define image data generators with data augmentation for training and normalization for validation/testing
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Randomly rotate images
    width_shift_range=0.1,    # Randomly shift images horizontally
    height_shift_range=0.1,   # Randomly shift images vertically
    shear_range=0.1,          # Apply shear transformation
    zoom_range=0.1,           # Apply zoom transformation
    horizontal_flip=True,     # Flip images horizontally
    fill_mode='nearest'       # Fill newly created pixels after rotation or shifting
)

In [6]:
test_val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale pixel values to [0, 1]


In [7]:
# Define batch size and image size
batch_size = 32
image_size = (224, 224)

In [8]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 2459 images belonging to 2 classes.


In [9]:
test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 690 images belonging to 2 classes.


In [10]:
val_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 355 images belonging to 2 classes.


In [ ]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.applications.DenseNet201(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

tb_callback = tf.keras.callbacks.TensorBoard(log_dir="/home/jaiswal.sh/Tensorboard-model-22072023/logs/SGD", histogram_freq=1)

# Compile the model
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model or fit the model

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[tb_callback]
)


2023-07-23 10:17:04.410627: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10


2023-07-23 10:17:14.920550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


77/77 [==============================] - ETA: 0s - loss: 0.1706 - accuracy: 0.9378 

2023-07-23 10:44:16.648373: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


77/77 [==============================] - 1675s 21s/step - loss: 0.1706 - accuracy: 0.9378 - val_loss: 0.2373 - val_accuracy: 0.9239
Epoch 2/10
77/77 [==============================] - 1627s 21s/step - loss: 0.0459 - accuracy: 0.9890 - val_loss: 0.1098 - val_accuracy: 0.9521
Epoch 3/10
77/77 [==============================] - 1624s 21s/step - loss: 0.0109 - accuracy: 0.9967 - val_loss: 0.0308 - val_accuracy: 0.9887
Epoch 7/10
77/77 [==============================] - 1625s 21s/step - loss: 0.0047 - accuracy: 0.9984 - val_loss: 0.0119 - val_accuracy: 0.9972
Epoch 8/10
77/77 [==============================] - 1603s 21s/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 9/10
54/77 [====================>.........] - ETA: 7:38 - loss: 0.0042 - accuracy: 0.9988

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit